<a href="https://colab.research.google.com/github/Suchit027/PCAP-Lab/blob/main/lab%205/add_q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%%writefile add_q2.cu
#include <stdio.h>
#include <stdlib.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

__global__ void sort(int *arr, int threadlimit, int blocklimit, int n){
  int t = threadIdx.x, b = blockIdx.x;
  if(t < threadlimit && b < blocklimit){
    int index = ((b * threadlimit) + t) * n;
      for(int i = 0; i < n; ++i){
        for(int j = i + 1; j < n; ++j){
          if(arr[index + i] > arr[index + j]){
            int t = arr[index + i];
            arr[index + i] = arr[index + j];
            arr[index + j] = t;
          }
        }
      }
  }
  return;
}

int main(){
  int *arr, m, n;
  printf("enter rows and cols\n");
  scanf("%d %d", &m, &n);
  printf("enter the elements\n");
  arr = (int *)malloc(m * n * sizeof(int));
  for(int i = 0; i < m; ++i){
    for(int j = 0; j < n; ++j){
      scanf("%d", &arr[(i * n) + j]);
    }
  }
  int *da;
  cudaMalloc((void **)&da, m * n * sizeof(int));
  int threadlimit, blocklimit;
  printf("enter threadlimit\n");
  scanf("%d", &threadlimit);
  if(m % threadlimit == 0){
    blocklimit = m / threadlimit;
  }
  else{
    blocklimit = (m / threadlimit) + 1;
  }
  cudaMemcpy(da, arr, m * n * sizeof(int), cudaMemcpyHostToDevice);
  sort<<<blocklimit, threadlimit>>>(da, threadlimit, blocklimit, n);
  cudaMemcpy(arr, da, m * n * sizeof(int), cudaMemcpyDeviceToHost);
  for(int i = 0; i < m; ++i){
    for(int j = 0; j < n; ++j){
      printf("%d ", arr[(i * n) + j]);
    }
    printf("\n");
  }
  cudaFree(da);
  free(arr);
  return 0;
}

Overwriting add_q2.cu


In [7]:
!nvcc -arch=compute_75 -code=sm_75 add_q2.cu -o add_q2.out


In [8]:
!./add_q2.out

enter rows and cols
3 3
enter the elements
1 2 3
3 2 1
1 2 3
enter threadlimit
1
1 2 3 
1 2 3 
1 2 3 
